# Define functions and preprocess data

In [1]:
import numpy as np
from sklearn import preprocessing
import math
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

import  sklearn.preprocessing


%matplotlib inline
# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()
    
    
# Remove all rows where the specified column is +/- sd standard deviations    
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)    
    
    
# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low
    
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd    
    
def get_volume(train_df):
    encode_text_dummy(train_df, "shape") # Get shape into dummies
    for value in ["height","width","length"]:
        train_df[value]=train_df[value].astype("float")
    train_df["volume"]=train_df['shape-box']*train_df['width']*train_df['height']*train_df['length']+0.25*train_df['shape-cylinder']*math.pi*(train_df['width']**2)*train_df['height']+train_df['shape-sphere']*math.pi*(train_df['length']**3)/6.0    
    for value in ["height","width","length"]:
        train_df.drop(value,axis=1,inplace=True)
    encode_text_dummy(train_df,"metal")
    for dummy in ['bronze', 'gold', 'platinum', 'silver', 'tin']:
        train_df["metal-"+dummy]=train_df["metal-"+dummy]*train_df["volume"]
    return train_df        


#Define the preprocess function, which includes the get_volume
def preprocess(train_df,columns=[ 'led', 'gears', 'motors',
       'shape-box', 'shape-cylinder', 'shape-sphere', 'volume', 'metal-bronze', 'metal-gold',
       'metal-platinum', 'metal-silver', 'metal-tin'],test=False,zscore=False):
    train_df=train_df.copy()
    if "metal_cost" in train_df.columns:
        train_df.drop("metal_cost",axis=1,inplace=True)
    encode_text_dummy(train_df, "shape")
    for value in ["height","width","length"]:
        train_df[value]=train_df[value].astype("float")
    train_df["volume"]=train_df['shape-box']*train_df['width']*train_df['height']*train_df['length']+0.25*train_df['shape-cylinder']*math.pi*(train_df['width']**2)*train_df['height']+train_df['shape-sphere']*math.pi*(train_df['length']**3)/6.0    
    for value in ["height","width","length"]:
        train_df.drop(value,axis=1,inplace=True)
    encode_text_dummy(train_df, "metal")
    train_df["led_vol"]=train_df["led"]*0.027 # encode the led_vol
    for dummy in ['bronze', 'gold', 'platinum', 'silver', 'tin']:
        train_df["metal-"+dummy]=train_df["metal-"+dummy]*train_df["volume"] 
    if test==False:
        train_y=train_df["weight"].copy()
    #train_df.drop("weight",axis=1,inplace=True)
        train_x=train_df[columns].copy()    
        if zscore==True:
            for column in train_x.columns:
                train_x[column]=train_x[column].astype("float")
                encode_numeric_zscore(train_x,column)
        x_train=train_x.values
        y_train=train_y.values       
            
        return(x_train,y_train)
    else:
        test_x=train_df[columns].copy()
        if zscore==True:
            for column in test_x.columns:
                test_x[column]=test_x[column].astype("float")
                encode_numeric_zscore(test_x,column)
        x_test=test_x.values        
        return(x_test)            
    
train_file="train.csv"
test_file="test.csv"
% matplotlib inline

In [2]:
train_df=pd.read_csv(train_file,na_values="?")
test_df=pd.read_csv(test_file,na_values="?")
#Read the data


In [3]:
test_df.head()

,id,shape,metal,metal_cost,height,width,length,led,gears,motors,led_vol,motor_vol,gear_vol,volume_parts,cost
0,0,cylinder,gold,39.10,4,8,0,10,16,14,0.27,NaN,NaN,NaN,NaN
1,1,box,platinum,29.44,4,5,8,42,5,4,NaN,NaN,NaN,NaN,96386.0
2,2,sphere,platinum,29.44,0,0,9,55,30,11,NaN,NaN,NaN,NaN,NaN
3,3,cylinder,platinum,29.44,9,7,0,67,22,8,NaN,NaN,NaN,NaN,NaN
4,4,box,bronze,0.05,3,9,3,34,46,0,NaN,NaN,NaN,NaN,91.0


In [37]:
train_df.columns

Index(['id', 'shape', 'metal', 'metal_cost', 'height', 'width', 'length',
       'led', 'gears', 'motors', 'led_vol', 'motor_vol', 'gear_vol',
       'volume_parts', 'cost', 'weight'],
      dtype='object')

In [4]:
train_df.shape

(879004, 16)

In [8]:
train_df["cost"].isna().value_counts() # This means that about 25% of the "cost" feature is missing

False    658670
True     220334
Name: cost, dtype: int64

In [9]:
test_df["cost"].isna().value_counts() # This means that about 25% of the "cost" feature is missing

False    74820
True     25180
Name: cost, dtype: int64

Preprocess to get the training and validation dataset

In [10]:
x_train,y_train=preprocess(train_df,columns=[ 'led', 'gears', 'motors',#'shape-box', 'shape-cylinder', 'shape-sphere',
        'metal-bronze', 'metal-gold',
       'metal-platinum', 'metal-silver', 'metal-tin'],zscore=True)
x_test=preprocess(test_df,columns=[ 'led', 'gears', 'motors',#'shape-box', 'shape-cylinder', 'shape-sphere',
         'metal-bronze', 'metal-gold',
       'metal-platinum', 'metal-silver', 'metal-tin'],test=True,zscore=True)

size=x_train.shape[0]
val = 0.2
x_val = x_train[:int(size*val),:].copy()
x=x_train[int(size*val):,:].copy()
y_val=y_train[:int(size*val)].copy()
y=y_train[int(size*val):].copy()

# Training models
We have tried several models, which with no improvment may not be shown here. We only list some models here for example.

In [17]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation,Dropout
y = y.ravel()

model_1 = Sequential()
model_1.add(Dense(128,input_dim=x.shape[1],activation="relu"))
model_1.add(Dense(1))
model_1.compile(loss='mean_squared_error', optimizer='adam')
model_1.fit(x, y,validation_data=(x_val,y_val),verbose=1,batch_size=128,epochs=6)  


Train on 703204 samples, validate on 175800 samples
Epoch 1/6
703204/703204 [==============================] - 15s 22us/step - loss: 3773765.7051 - val_loss: 388210.9890
Epoch 2/6
703204/703204 [==============================] - 15s 22us/step - loss: 210505.7475 - val_loss: 175274.1698
Epoch 3/6
703204/703204 [==============================] - 16s 23us/step - loss: 172160.8386 - val_loss: 167666.3174
Epoch 4/6
703204/703204 [==============================] - 16s 23us/step - loss: 167947.4966 - val_loss: 165628.2733
Epoch 5/6
703204/703204 [==============================] - 16s 23us/step - loss: 166439.3198 - val_loss: 164612.7307
Epoch 6/6
703204/703204 [==============================] - 16s 23us/step - loss: 165628.9374 - val_loss: 163846.8490


In [18]:
model_1.evaluate(x_val,y_val)

175800/175800 [==============================] - 5s 27us/step


163846.84883176905

In [16]:
y_test=model_1.predict(x_test)

The hyper parameters are adjusted. The best fitted predicted data using Neural Network is saved as one file. Similarly, those using Randomforest, GradientBoostTree too. Those out come would be merged at last. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics



from sklearn.model_selection import cross_val_score

## RandomForest
rfr = RandomForestRegressor(n_estimators=100,max_features='sqrt')
rfr.fit(x, y)
rfr_y_predict = rfr.predict(x_val)
MSE=mean_squared_error(rfr_y_predict,y_val)
print("RFR")
print(MSE)

## GradientBoostRegressor
gbr=GradientBoostingRegressor(loss="ls",n_estimators=400)
gbr.fit(x, y)
gbr_y_predict = gbr.predict(x_val)
MSE=mean_squared_error(gbr_y_predict,y_val)
print{"GBRT"}
print(MSE)







## sample merge

In [ ]:
a = pd.read_csv("rfr_150.csv")
b = pd.read_csv("NN_2Layer.csv")



submit_df = pd.DataFrame()
submit_df["id"] = test_df["id"]
weight = (b["weight"]+ a["weight"] )/2.0

submit_df["weight"] = weight
submit_df.to_csv("merged.csv",index = False)

## During trying models and features, we found(ba masking) that datapoints with existing "cost" could super precisely predicted via several features including cost. which in our test only behave best in random forest

In [20]:
mask = train_df["cost"].notna()

In [21]:
mask_df = train_df[mask]

In [22]:
mask_df = get_volume(mask_df)

C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [29]:
size = mask_df.shape[0]
columns = ["cost",'led', 'gears', 'motors',
       'volume', 'metal-bronze', 'metal-gold',
       'metal-platinum', 'metal-silver', 'metal-tin']
x_train = mask_df[columns].copy()
y_train  = mask_df["weight"].copy()
val = 0.1
x_val = x_train[:int(size*val)].copy()
x=x_train[int(size*val):].copy()
y_val=y_train[:int(size*val)].copy()
y=y_train[int(size*val):].copy()

In [32]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
## RandomForest
rfr = RandomForestRegressor(n_estimators=150,max_features='sqrt')
rfr.fit(x, y)
rfr_y_predict = rfr.predict(x_val)
MSE=mean_squared_error(rfr_y_predict,y_val)
print("RFR")
print(MSE)




C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refac

RFR
1469.1833051839415


That validation error is so perfect! I guess if we fill the nan values in cost, we would achieve great outcome! But the bad news is that finally we did not make it to predic cost precisely. Even we tried several models and features, or devide according to the metal type and shape and conquer.

## Finally, we predicted the test data using this model with missing cost filled as mean(or poorly predicted value). And the substitute the rows with missing cost with the merged outcome predicted before.

In [39]:

mask = test_df["cost"].isna()

a = pd.read_csv("merged.csv")
b = pd.read_csv("cost_150.csv")



submit_df = pd.DataFrame()
submit_df["id"] = test_df["id"]
b["weight"][mask] = a["weight"][mask]
submit_df["weight"] = b["weight"]


C:\Users\Gang\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
submit_df.to_csv("merge_1759.csv",index = False)

# Computing the volume of each motor, gear using linear regression
### We computed the volume via linear regression in order to fiil in the missing values, which we used to predict the weight as well as predict cost. But the improvement was minimal.

In [33]:
density = {"bronze":9.29,"gold":19.32,"platinum":20.09,"silver":10.49,"tin":7.31}

weight method

In [ ]:
from sklearn import linear_model
led = []
gears = []
motors = []
for metal in density.keys():
    mask = train_df["metal-"+metal]!=0
    mask_df = train_df[mask]
    x = mask_df[["volume","led","gears","motors"]] .copy()
    y = mask_df["weight"].copy()
    
    regr = linear_model.LinearRegression()
    regr.fit(x,y)
    led.append(regr.coef_[1])
    gears.append(regr.coef_[2])
    motors.append(regr.coef_[3])
    

In [ ]:
x=np.array(list(density.values()))
y= np.array(gears)
x=x.reshape(-1, 1)
regr = linear_model.LinearRegression()
regr.fit(x,y)
regr.coef_ ## The slope should be the volume

In [ ]:
#we get
gears_vol = 0.00112197
motors_vol = 12.18045214
led_vol = 0.027

cost method

In [ ]:
mask1=train_df["cost"].notna()
mask1_df = train_df[mask1]

In [ ]:
from sklearn import linear_model
led = []
gears = []
motors = []
for metal in density.keys():
    mask = mask1_df["metal-"+metal]!=0
    mask_df = mask1_df[mask]
    x = mask_df[["volume","led","gears","motors"]] .copy()
    y = mask_df["cost"].copy()
    
    regr = linear_model.LinearRegression()
    regr.fit(x,y)
    led.append(regr.coef_[1])
    gears.append(regr.coef_[2])
    motors.append(regr.coef_[3])
    

In [ ]:
metalcost={"bronze":0.05,"gold":39.1,"platinum":29.44,"silver":0.47,"tin":0.06}

In [ ]:
led_=np.array(led)
metal_cost=np.array(list(metalcost.values()))
ledcoef=led/metal_cost
ledcoef

gears=np.array(gears)
metal_cost=np.array(list(metalcost.values()))
gearscoef=gears/metal_cost

gearscoef

motors=np.array(motors)
metal_cost=np.array(list(metalcost.values()))
motorscoef=motors/metal_cost

motorscoef

In [ ]:
x=np.array(list(density.values()))
y= np.array(motorscoef)
x=x.reshape(-1, 1)
regr = linear_model.LinearRegression()
regr.fit(x,y)
regr.coef_ ## The slope should be the volume

some use of the feature

In [ ]:
train_df['motor_vol'] = ((12.18045214+13.49438265)/2.0) * train_df['motors']
train_df['gear_vol'] = (0.64987203/2.0) * train_df['gears']
train_df["led_vol"]=train_df["led"]*0.027
train_df['volume_parts'] = train_df['led_vol'] + train_df['motor_vol'] + train_df['gear_vol']


mask3=train_df["volume_parts"]>train_df["volume"]
train_df["volume_parts"][mask3]=0
train_df["led"][mask3]=0
train_df["gears"][mask3]=0
train_df["motors"][mask3]=0



In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
gbr=GradientBoostingRegressor(loss="ls",n_estimators=400)
gbr.fit(x, y)
gbr_y_predict = gbr.predict(x_test)
MSE=mean_squared_error(gbr_y_predict,y_test)
MSE


In [ ]:
rfr1 = RandomForestRegressor(n_estimators=100,max_features='sqrt')